In [319]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [ ]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"DXPE"}, 
             {'symbol':"TSM"}, 
             {'symbol':"WLDN"}, 
             {'symbol':"SSRM"}, 
             {'symbol':"LRN", 'basis':137.753}, 
             {'symbol':"UNFI", 'basis':20.752}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':100.482}, 
             {'symbol':"EZPW", 'basis':15.460}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':60.671}, 
             {'symbol':"ITRN", 'basis':35.710}, 
             {'symbol':"CRDO", 'basis':0.010}, 
             {'symbol':"PYPL", 'basis':67.980}, 
             {'symbol':"ALL", 'basis':194.927}, 
             {'symbol':"LC", 'basis':8.752}, 
             {'symbol':"QTWO"}, 
             {'symbol':"CLS", 'basis':0.010}, 
             {'symbol':"CCL", 'basis':8.288}, 
             {'symbol':"AGX", 'basis':0.010}, 
             {'symbol':"POWL", 'basis':164.655}, 
             {'symbol':"PPC", 'basis':45.089}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE", 'basis':115.143}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':65.040}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA", 'basis':97.398}, 
             {'symbol':"TWLO", 'basis':87.178}, 
             {'symbol':"PEP"}, 
             {'symbol':"APP", 'basis':221.323}, 
             {'symbol':"TMUS", 'basis':218.823}, 
             {'symbol':"STRL"}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':40.612}, 
             {'symbol':"CAAP", 'basis':21.570}
             ]

# Brokerage
brokerage_funds = [{'symbol':"FRSH", 'basis':14.000}, 
                   {'symbol':"PGY"}, 
                   {'symbol':"COMM"}, 
                   {'symbol':'FINV', 'basis':8.950}, 
                   {'symbol':"LX", 'basis':6.629}, 
                   {'symbol':"WLDN"}, 
                   {'symbol':'BCS', 'basis':18.569}, 
                   {'symbol':'PUK'}, 
                   {'symbol':'PSIX'}, 
                   {'symbol':'NGD', 'basis':4.400}, 
                   {'symbol':'GFI'}, 
                   {'symbol':"CRDO", 'basis':0.010}, 
                   {'symbol':'BKTI', 'basis':40.380}, 
                   {'symbol':"SSRM"}, 
                   {'symbol':"UNFI", 'basis':22.716}, 
                   {'symbol':"MFC"}, 
                   {'symbol':"EZPW", 'basis':15.773}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"WFC", 'basis':66.893}, 
                   {'symbol':"ITRN"}, 
                   {'symbol':"PYPL", 'basis':68.030}, 
                   {'symbol':"LC"}, 
                   {'symbol':"CCL", 'basis':0.010}, 
                   {'symbol':"PPC", 'basis':44.580}, 
                   {'symbol':"SYF"}, 
                   {'symbol':"ATGE", 'basis':86.085}, 
                   {'symbol':"SKYW", 'basis':74.583}, 
                   {'symbol':"BLBD"}, 
                   {'symbol':"GM"}, 
                   {'symbol':"OKTA", 'basis':99.535}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   {'symbol':"GRBK"}, 
                   {'symbol':"UBER"}, 
                   {'symbol':"CAAP"},
                   {'symbol':"NVDA", 'basis':58.908},
                   {'symbol':"QUBT", 'basis':16.042},
                   {'symbol':"RGTI"}, 
                   {'symbol':"FBTC", 'basis':65.965, 'is_etf': True}
                   ]

In [321]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [322]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=.99, vwap_days=21)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [323]:
brokerage_df[brokerage_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
3,FINV,8.950,8.73,9.52,19.87,10.06,6.37,1.039,0.0310,Hold
4,LX,6.629,6.40,6.73,17.80,7.23,5.93,0.748,0.0261,Hold
6,BCS,18.569,20.04,18.47,45.74,18.93,9.66,0.681,0.0043,Sell
9,NGD,4.400,4.30,4.46,2.97,4.70,24.29,2.967,0.0000,Hold
11,CRDO,0.010,109.38,92.67,6.58,98.98,348.23,27.180,0.0000,Sell
12,BKTI,40.380,40.14,41.60,29.65,45.05,14.56,4.316,0.0000,Hold
14,UNFI,22.716,27.69,24.52,24.52,25.88,inf,1.051,0.0000,Sell
16,EZPW,15.773,13.13,13.55,23.53,14.08,11.18,0.955,0.0000,Hold
17,ARQT,15.996,14.99,14.08,14.08,14.95,inf,12.500,0.0000,Sell
18,WFC,66.893,82.62,79.68,99.65,82.26,14.50,1.626,0.0190,Sell


In [324]:
brokerage_df[brokerage_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,FRSH,0.0,13.91,13.98,13.98,14.59,inf,3.890,0.0000,Hold
1,PGY,0.0,29.08,26.06,26.06,28.44,inf,7.020,0.0000,Sell
2,COMM,0.0,8.34,7.71,7.71,8.21,inf,30.360,0.0000,Sell
5,WLDN,0.0,83.42,71.90,31.76,75.92,48.83,4.844,0.0000,Sell
7,PUK,0.0,25.05,24.45,27.95,24.99,15.00,1.836,0.0092,Sell
8,PSIX,0.0,89.06,72.50,21.06,81.71,25.23,24.300,0.0000,Sell
10,GFI,0.0,25.31,23.67,17.05,24.67,17.84,4.237,0.0221,Sell
13,SSRM,0.0,12.27,12.22,14.90,12.85,29.78,0.781,0.0000,Hold
15,MFC,0.0,31.48,30.46,35.93,31.24,16.13,1.632,0.0524,Sell
19,ITRN,0.0,39.78,38.55,29.57,40.04,14.62,4.246,0.0491,Hold


In [325]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=.95, vwap_days=63)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [326]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
4,LRN,137.753,128.56,135.44,84.51,148.50,20.13,3.943,0.0000,Hold
5,UNFI,20.752,27.69,23.39,23.39,25.90,inf,1.051,0.0000,Sell
7,EAT,100.482,159.50,148.41,38.90,163.04,21.89,26.340,0.0000,Hold
8,EZPW,15.460,13.31,13.38,23.85,14.51,11.18,0.955,0.0000,Hold
9,ARQT,15.932,14.99,13.05,13.05,14.50,inf,12.500,0.0000,Sell
10,WFC,60.671,82.62,72.54,99.65,77.96,14.50,1.626,0.0190,Sell
11,ITRN,35.710,39.78,35.31,29.57,38.33,14.62,4.246,0.0491,Sell
12,CRDO,0.010,109.38,71.51,6.58,80.10,348.23,27.180,0.0000,Sell
13,PYPL,67.980,71.45,67.62,51.49,73.11,17.58,3.756,0.0000,Hold
14,ALL,194.927,194.03,186.80,195.23,200.73,13.11,2.584,0.0192,Hold


In [327]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,DXPE,0.0,110.53,84.75,67.47,93.33,23.33,3.945,0.0000,Sell
1,TSM,0.0,241.33,196.64,93.22,211.39,28.66,8.020,0.0136,Sell
2,WLDN,0.0,83.42,59.68,31.76,65.36,48.83,4.844,0.0000,Sell
3,SSRM,0.0,12.27,11.24,14.90,12.35,29.78,0.781,0.0000,Hold
6,MFC,0.0,31.48,29.53,35.93,31.64,16.13,1.632,0.0524,Hold
16,QTWO,0.0,90.38,83.49,83.49,90.56,inf,10.370,0.0000,Hold
22,SYF,0.0,72.87,58.50,109.25,63.02,8.85,1.724,0.0144,Sell
24,BRK-B,0.0,476.86,468.02,618.44,499.54,12.84,1.588,0.0000,Hold
25,SFM,0.0,156.94,152.97,44.06,167.47,35.87,12.130,0.0000,Hold
27,BLBD,0.0,45.26,38.58,25.32,41.90,14.32,7.650,0.0000,Sell


In [328]:
# Research new investments
watch_list = [{'symbol':'BSAC'}, # Chile - Latin America - Financials
              {'symbol':'KARO'}, # Singapore - Technology - SaaS
              {'symbol':'QFIN'}, # China - FinTech
              {'symbol':'TSM'}, # Taiwan - Semiconductors
              {'symbol':'CCU'}, # Chile - Consumer Staples - beverage distributor
              {'symbol':'ABEV'}, # Brazil - Financials - but distributes beers?
              {'symbol':'TIMB'}, # Brazil - Communications - Telecom (mobile networking)
              {'symbol':'ASR'}, # Mexico - Industrials - Airport operations in South America
              {'symbol':'RYAAY'}, # Ireland - Industrials - budget airline
              {'symbol':'AMX'}, # Mexico - Communications - mobile network
              {'symbol':'ERIC'}, # Sweden - Communications
              {'symbol':'RDWR'}, # Sweden - Communications
              {'symbol':'PGY'}, # Sweden - Communications
              {'symbol':'STX'}, # Ireland - Technology
              {'symbol':'NVTS'} # Ireland - Technology
             ] 
watch_table = build_analysis_table(watch_list, api_key, margin_of_safety=1, vwap_days=63)
internationals = build_analysis_table(watch_list, api_key, margin_of_safety=0.95, vwap_days=126)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

internationals_df = pd.DataFrame(internationals, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [329]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,23.30,24.17,28.58,24.66,9.83,2.319,0.0584,Hold
1,KARO,0,46.81,48.43,18.88,51.17,27.53,7.660,0.4590,Hold
2,QFIN,0,35.04,41.07,72.96,42.75,5.41,1.462,0.0370,Hold
3,TSM,0,241.33,206.99,93.22,211.39,28.66,8.020,0.0136,Sell
4,CCU,0,11.71,13.37,16.69,13.72,12.53,1.348,0.0381,Hold
5,ABEV,0,2.34,2.41,2.49,2.47,14.38,2.112,0.0664,Hold
6,TIMB,0,17.91,18.20,20.20,18.60,13.78,1.956,0.0866,Hold
7,ASR,0,303.92,312.97,286.72,320.68,12.47,3.090,0.0847,Hold
8,RYAAY,0,62.88,55.27,50.33,56.48,14.00,3.824,0.0170,Sell
9,AMX,0,18.27,17.15,15.83,17.52,19.98,2.287,0.0281,Sell


In [330]:
internationals_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,23.30,22.09,28.58,23.79,9.83,2.319,0.0584,Hold
1,KARO,0,46.81,44.32,18.88,49.26,27.53,7.660,0.4590,Hold
2,QFIN,0,35.04,38.65,72.96,42.83,5.41,1.462,0.0370,Hold
3,TSM,0,242.75,175.79,93.77,191.76,28.66,8.020,0.0136,Sell
4,CCU,0,11.71,13.06,16.69,14.16,12.53,1.348,0.0381,Hold
5,ABEV,0,2.34,2.16,2.49,2.33,14.38,2.112,0.0664,Sell
6,TIMB,0,17.91,15.32,20.20,16.49,13.78,1.956,0.0866,Sell
7,ASR,0,303.92,282.55,286.72,306.37,12.47,3.090,0.0847,Hold
8,RYAAY,0,62.88,47.56,50.33,51.34,14.00,3.824,0.0170,Sell
9,AMX,0,18.27,15.09,15.83,16.28,19.98,2.287,0.0281,Sell


In [331]:
# Research new investments
mag7_test = [{'symbol':'MSFT'}, 
             {'symbol':'AAPL'}, 
             {'symbol':'NVDA'}, 
             {'symbol':'AMZN'}, 
             {'symbol':'GOOG'}, 
             {'symbol':'GOOGL'}, 
             {'symbol':'META'}, 
             {'symbol':'BRK-B'}, 
             {'symbol':'TSLA'}
             ] 
mag7_test_table = build_analysis_table(mag7_test, api_key, margin_of_safety=0.95, vwap_days=63)

# Create watch list data frame
mag7_df = pd.DataFrame(mag7_test_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [332]:
mag7_test_table

[['MSFT', 0, 512.57, 443.03, 138.72, 473.06, 39.58, 11.83, 0.0063, 'Sell'],
 ['AAPL', 0, 211.27, 192.96, 30.99, 207.24, 33.34, 47.82, 0.0047, 'Sell'],
 ['NVDA', 0, 175.51, 134.34, 19.35, 145.04, 55.93, 50.47, 0.0002, 'Sell'],
 ['AMZN', 0, 231.01, 198.38, 77.47, 213.21, 37.98, 8.03, 0, 'Sell'],
 ['GOOG', 0, 196.43, 162.89, 99.7, 176.45, 20.64, 6.45, 0.0052, 'Sell'],
 ['GOOGL', 0, 195.75, 161.74, 99.75, 175.31, 20.51, 6.44, 0.0052, 'Sell'],
 ['META', 0, 700.0, 625.92, 247.93, 674.08, 28.04, 9.75, 0.0028, 'Sell'],
 ['BRK-B', 0, 476.86, 468.02, 618.44, 499.54, 12.84, 1.588, 0, 'Hold'],
 ['TSLA', 0, 321.2, 296.06, 36.67, 326.77, 193.8, 13.58, 0, 'Hold']]

In [333]:
mag7_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,MSFT,0,512.57,443.03,138.72,473.06,39.58,11.830,0.0063,Sell
1,AAPL,0,211.27,192.96,30.99,207.24,33.34,47.820,0.0047,Sell
2,NVDA,0,175.51,134.34,19.35,145.04,55.93,50.470,0.0002,Sell
3,AMZN,0,231.01,198.38,77.47,213.21,37.98,8.030,0.0000,Sell
4,GOOG,0,196.43,162.89,99.70,176.45,20.64,6.450,0.0052,Sell
5,GOOGL,0,195.75,161.74,99.75,175.31,20.51,6.440,0.0052,Sell
6,META,0,700.00,625.92,247.93,674.08,28.04,9.750,0.0028,Sell
7,BRK-B,0,476.86,468.02,618.44,499.54,12.84,1.588,0.0000,Hold
8,TSLA,0,321.20,296.06,36.67,326.77,193.80,13.580,0.0000,Hold


In [334]:
# Machine Learning

In [335]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [336]:
# Feature selection and normalization

In [337]:
# Build model

In [338]:
# Train model

In [339]:
# Evaluate model

In [340]:
# Refine model through hyperparameter tuning